<a href="https://colab.research.google.com/github/br4bit/Neural-Network-Training/blob/master/Training_Deep_Neural_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Addestramento rete neurale profonda (Deep Neural Network)

Questo tipo di addestramento richiede un'analisi accurata dei seguenti problemi:



*  Primo: Problema del *Vanishing Gradients* e il correlato *Exploding Gradients* che si presenta nelle reti neurali molto profonde (più di 10 layers), questo problema rende difficile l'allenare i layers che si trovano nello strato inferiore.
*   Secondo: L'addestramento della rete neurale rallenta di molto con reti molto grandi.

*  Terzo: Un modello con milioni di parametri rischirebbe gravemente l'overfitting sul set di training.



# Problemi del Vanishing/Exploding Gradients

L'algoritmo di backpropagation lavora andando dall'output layer all'input layer, propagando l'errore del gradiente durante il tragitto. Una volta che l'algoritmo ha calcolato il gradiente della cost function utilizzando ogni parametro della rete, i parametri della rete vengono aggiornati (Gradient Descent step).

Sfortunatamente i gradienti calcolati diventano via via, sempre più piccoli man mano che l'algoritmo procede nei layers inferiori. Come risultato l'aggiornamento del Gradiente Descent, lascia i pesi delle connessioni nel layer inferiore virtalmente immutati (non li cambia), e l'addestramento non converge quindi ad una buona soluzione, da qui il nome *Vanishing Gradients*. In molti casi però, può accadere il contrario, ovvero i gradienti (derivate parziali) possono diventare molto grandi, e quindi causare anche l'incremento dei pesi in alcuni layer, di conseguenza l'algoritmo diverge (*Exploding Gradients*).
Più in generale le reti neurali, soffrono di gradienti instabili, dove diversi layers possono imparare a velocità ampliamente differenti. Questo è uno dei motivi per cui sono state abbandonate per molto tempo.

Ma nel 2010 grazie a : [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

Che attraverso dei test con le funzioni di attivazione e inizializzazione dei pesi utilizzando una distribuzione normale con media 0 e deviazione standard a 1. In breve è stato dimostrato che con quel tipo di funzione di attivazione e schema di inizializzazione, la varianza dell'output di ogni layer era molto più grande della varianza dei proprio inputs. Andando in avanti nella rete la varianza continuava a crescere ad ogni layer fino a che la funzione di attivazione assumeva valori di saturazione nei layer in cima. Questo in realtà è aggravato dal fatto che la funzione logisitica ha una media di 0.5, non 0! (ad esempio la funzione tangente hyperbolica ha media 0 e si comporta molto bene rispetto alla logistic nelle reti molto grandi).

Diamo un'occhiata alla funzione logistica di attivazione (Sigmoid):

![alt text](https://www.simplilearn.com/ice9/free_resources_article_thumb/gradients-in-sigmoid-activation-functions.jpg)

E' possibile vedere che quando l'input assume valori molto grandi (sia positivi che negativi), la funzione satura o meglio assume valore 1 o 0, e in quei punti la pendenza della retta tangente (la derivata) assume valori estremamente vicino allo 0.
Ne consegue che durante la fase di backpropagation, non esiste virtualmente nessun gradiente da propagare attraverso la rete, e quel piccolo gradiente che esiste si diluisce, mentre il backpropagation progredisce verso il basso attraverso i layers superiori, quindi non rimane nulla per i livelli inferiori.

## Xavier and He Inizializzazione

Dall'articolo Glorot e Bengio hanno proposto un modo per alleviare significamente questo problema. C'è bisogno che il segnale fluisca in modo appropriato in entrambe le direzioni: In avanti quando si fa una predizione e indietro quando si backpropagano i gradienti. Il segnale quindi non deve morire o assumere valori di saturazione.

Per far si che il segnale fluisca in modo corretto, l'autore ci dice che la varianza degli output di ogni layer deve essere uguale alla varianza degli input del layer : Var_Out(layer[1] = Var_In(layer[1])), questo per ogni layer. Inoltre ce bisogno che i gradienti abbiano prima e dopo l'attraversamento della rete nella direzione opposta, la stessa varianza. Non è possibile però garantire entrambi le condizioni a meno che i layers non abbiano lo stesso numero di connessioni di input e output, ma è possibile arrivare ad un compromesso che funziona molto bene in pratica:

Le connessioni dei pesi (matrice W) devono essere inizializzati in modo casuale secondo l'equazione di Xavier o Glorot.

![alt text](https://www.simplilearn.com/ice9/free_resources_article_thumb/normal-distribution-with-0-mean-and%20standard-deviation-formula.jpg)

Usando una delle due inizializzazione è possibile velocizzare l'addestramento. Inoltre questa strategia di inizializzazione nel caso si decida di utilizzare la funzione ReLU come funzione di attivazione è chiamata *He Initialization*.

Di default la funzione *tf.layers.dense()* usa la Xavier con distribuzione uniforme, è possibile cambiare anche con l'inizializzazione di He usando *variance_scaling_initializer()*

In [0]:
he_init = tf.contrib.layers.variance_scaling_initializer()

hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu(), 
                          kernel_initializer=he_init, name="hidden1")


## Non Saturazione della funzione di attivazione

Sfortunatamente la funzione di attivazione ReLU non è perfetta, ma soffre di un problema conosciuto come *dying ReLus*, durante l'addestramento (training), alcuni neuroni è come se morissero, o meglio smettono di dare come output valori che sono diversi da 0, specialmente se si usa un learning rate molto alto. 

Per risolvere questo problema si usano delle varianti della funzione ReLU, come la *leaky ReLU*, questa funzione è definita come:

![alt text](http://img.thothchildren.com/09d44cab-d699-402a-af31-fdced34d94fd.png)
![alt text](http://wangxinliu.com/images/machine_learning/leakyrelu.png)

Dove α è un Hyperparametro e definisce quanto deve perdere la funzione o meglio qual è la pendenza della funzione per x<0 e tipicamente è impostato a 0.01. Questa piccola pendenza ci assicura che la Leaky ReLU non muoia mai.

Esistono molto varianti di questo genere, in particolare una nuova funzione di attivazione chiamata ELU (Exponential linear Unit) che ha superato la performance di tutte le varianti di ReLU nei vari esperimenti: tempo di training ridotto e performance migliorata sul test test. 

Ecco la sua definizione e rappresentazione:

![alt text](https://i.gyazo.com/7851e87ec39ce6a960e26709f39c1825.png)

![alt text](https://blog.paperspace.com/content/images/2018/06/ELU.png)

E' molto simile alla ReLU ma con alcune differenze:

* Primo: Ha una regione di valori negativi quando x<0, che permette alle unità di assumere valori di output vicino allo 0, il che aiuta con il problema del vanishing gradient. Hyperparamentro  α definisce il valore che la ELU deve approcciare quando x è un numero negativo molto grande, usualmente è impostato a 1.
* Secondo: Il suo gradiente per x<0 non è zero, quindi si evita il problema dei neuroni morti.
* Terzo: Quando Hyperparametro è uguale a 1 la funzione è molto regolare/morbida ovunque, incluso nell'intorno di x=0, questo aiuta molto a velocizzare il calcolo dei gradienti, evitando saltelli a destra e a sinistra nell'intorno di x=0.

L'unica pecca è che la ELU impiega molto tempo computazionale, dovuto alla funzione esponenziale, ma questo tempo è compensato dal fatto che durante il training converge molto più velocemente, quindi una rete con ReLU risulta di poco più veloce.

Quindi, quale funzione di attivazione utilizzare per gli hidden layers?

In generale ELU > leaky RELU (e varianti) > ReLU > tanh > logistic.

Se ci interessa il tempo di performance durante il runtime, la leaky ReLU è preferibile rispetto alla ELU.

Ma se il tempo di performance non ci interessa molto e abbiamo potenza computazione elevata è possibile utilizzare altre funzioni nel caso in cui: RReLU (overfitting) oppure PReLU se abbiamo un training set molto grande.

TensorFlow ci offre la *elu()* che è possibile utilizzare per costruire il nostro modello di rete neurale. Semplicemente basta impostare l'argomento del campo *activation*.

In [0]:
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1")

Sfortunatamente TensorFlow non ha una funzione predefinita per la leaky ReLU, ma è possibile definirne una, nel seguente modo:

In [0]:
def leaky_relu(z, name=None):
  return tf.maximum(0.01 * z, z, name=name)

hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")

## Batch Normalization

Anche se usando l'inizializzazione dei pesi con il metodo di He, si ha una riduzione del problema del vanishing/exploding gradients all'inizio dell'addestramento, non è garantito che in seguito questo problema non si presenti, durante il training della rete.

Una tecnica scoperta nel 2015 chiamata *Batch Normalization (BN)* affronta il problema in modo diretto o più in generale, il problema della distribuzione di ogni input dei layer, che cambia durante il training a seconda di come cambiano i parametri dei layer precedenti, questo cambiamento della distribuzione è noto come *Internal Covariate Shift*.

La tecnica consiste nell'aggiungere un'operazione nel modello prima che venga eseguita la funzione di attivazione di ogni layer, semplicemente centrando a zero gli input e normalizzandoli, scalando e shiftando il risultato, usando due nuovi parametri per layer (uno per lo scaling e l'altro per lo shifting). In poche parole questa operazione permette al modello di imparare lo scale e la media ottimale degli inputs per ogni layer, in modo da rendere la distribuzione degli input più uniforme, riducendone la varianza o variazione.

L'algoritmo ha bisogno di conoscere per ogni mini-batch di input, la media e la deviazione standard.
Tutte le operazioni che esegue l'algoritmo:


>![alt text](https://i.gyazo.com/45070cbcbd577a47f40cb04034784fc9.png)



1. Calcolo della media  e della varianza degli input dei layer:
La media è calcolata su tutti i campioni del mini-batch B.

2. Varianza calcolata su tutto il mini-batch, dove mB è il numero dei campioni o istanze presenti nel mini-batch.

3. E' l'input normalizzato e centrato in zero.

4. Gamma è il parametro di scaling per il layer.

5. Beta è il parametro di shifting (offset) per il layer.

6. Epsilon è un numero arbitrario piccolo per evitare la divisione per zero (tipicamente 10-5) è chiamato *Smoothing Term*.

7. **z(i)** è l'output dell'operazione di Batch Normalization (BN): non è nient'altro che una versione shiftata e scalata degli input.

Durante la valutazione con il test set, non esiste nessun mini-batch, quindi si usa la media e la deviazione standard calcolate su tutto il training set. In totale si hanno quattro parametri per ogni batch normalizzato: gamma (scale), beta(offset), mu(media) e sigma (deviazione standard).

Con questa tecnica si riduce considerevolmente il problema del vanishing/exploding gradients al punto di permettere l'uso delle funzione di attivazioni che hanno problemi di saturazione come tanh,logist,ReLU,ecc.
La rete risulta inoltre meno sensibile all'inizializzazione dei pesi, ed è possibile usare anche un learning rate alto, velocizzando quindi il processo di addestramento (meno step e precisione più elevata).

Ma come tutte le tecniche viste fino ad ora, anche il Batch Normalization introduce una ulteriore complessità al modello, la penalizzazione in runtime: La rete neurale effetua delle predizioni lenti a causa della computazione extra richiesta per ogni layer, deve effettuare il BN (il primo layer quello di input non viene normalizzato, i dati vengono normalizzati a partire dal primo hidden layer), quindi se si ha bisogno di una rete che effettua delle predizioni veloci sui dati, prima di implementare il Batch Normalization è opportuno dare un'occhiata delle performance pianificando al meglio la ELU + He Initialization.

### Implementazione del Batch Normalization in TensorFlow

TensorFlow ci viene in aiuto anche in questo caso, mettendo a disposizione la funzione *tf.nn.batch_normalization()* che semplicemente centra e normalizza gli inputs, ma tocca a noi calcolare la media e la deviazione standard (basate sul mini-batch di dati corrente durante la fase di training oppure su tutto il dataset durante la fase di testing) e passarle come parametri a questa funzione, e bisogna anche gestire la creazione dei parametri di scaling e shifting(offset), passando anche questi alla funzione. **Non è un approccio molto conveniente**.

La funzione *tf.layers.batch_normalization()* si occupa di eseguire tutte queste operazioni.
Riprendiamo il modello della guida precedente, abbiamo:

In [0]:
import tensorflow as tf

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None,n_inputs), name="X")

training = tf.placeholder_with_default(False,shape=(), name='training')

#Costruzione del primo hidden layer e layer di BN
hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1) #output del layer normalizzato

#Costruzione del secondo hidden layer e layer di BN
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

#Costruzione del layer di output
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)




**Opzionale:** La prima parte del codice è quella vista precedetemente, fino ad arrivare alla definizione della variabile *training*, questa variabile sarà settata a True durante la fase di training, e serve per dire alla funzione *tf.layers.batch_normalization()* quando usare la deviazione standard e media calcolate sul mini-batch corrente durante la fase di training, e quando usare la media e deviazione standard dell'intero training set, durante la fase di testing.

Successivamente si costruiscono gli hidden layers e l'output layer.
Per gli hidden layes si nota che durante la chiamata della funzione *tf.layers.dense()* non è utilizzato il parametro *activation* questo perchè vogliamo applicare la funzione di attivazione soltanto dopo il batch normalization del layer. Creiamo i batch normalization layers con la funzione *tf.layers.batch_normalization()* impostando i parametri training e momentum. L'algoritmo di BN usa la tecnica del decadimento dell'esponenziale per calcolare le medie correnti, ecco perchè è richiesto anche il parametro *momentum*, dato un nuovo valore v, le medie correnti v' vengono aggiornate attraverso l'equazione:

v' <- v' * momentum + v x (1- momentum).

Un buono valore per il momentum è tipicamente vicino a 1 (0.9 , 0.99, 0.999), per dataset molto grandi e mini-batch piccoli è preferibile usare più 9 dopo lo 0.



---



Il codice risulta ripetitivo, con i parametri di BN che appaiono ovunque, per evitare questa ripetizione di linee di codice, usiamo la funzione *partial* dal modulo *functools* che crea un involucro attorno alla funzione e ci permette di usarla in modo semplice. Modificando così la creazione dei layers in questo modo:

In [0]:
from functools import partial

my_batch_norm_layer = partial (tf.layers.batch_normalization, training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1) #funzione di attivazione sul batch normalization

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

Se si hanno 10 layer e si vuole usare la stessa funzione di attivazione, inizializzazione, regolarizzazione su tutti i layer, questo piccolo trucchetto renderà il codice più leggibile.

Il resto di costruzione del modello è quello visto nella guida precedente: definizione della cost function e loss, creazione dell'optimizer e GradientDescent per minimizzare la cost function, definizione della valutazione del modello, creazione della variabile initializer (init), creazione del *Saver*.

La fase di esecuzione è molto simile, con due eccezioni.
Primo: ogni volta che durante la fase di training si esegue un'operazione che dipende dal layer *batch_normalization()*, bisogna impostare la variabile di training a True.
Secondo: la funzione di *batch_normalization()* crea delle operazioni che bisogna valutare ad ogni step durante il training in modo da aggiornare la media dei mini-batch. 
Queste operazioni sono automaticamente aggiunte  alla collezione *UPDATE_OPS* , tutto quello che bisogna fare e prendere la lista delle operazioni in quella collezione ed eseguirle ad ogni iterazione durante il training

In [0]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
  
  init.run()
  
  for epoch in range(n_epochs):
    
    for iteration in range(mnist.num_examples // batch_size):
    
      X_batch, y_batch = mnist.train.next_batch(batch_size)
      sess.run([training_op,extra_update_ops], feed_dict={training=True, X: X_batch, y: y_batch})
    
    accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
    print(epoch, "Test Accuracy: ",accuracy_val)
    save_path = saver.save(sess,"./my_model_final.ckpt")

## Gradient Clipping

Questa tecnica veniva utilizzata prima del BN e consiste nell'effettuare una operazione di "clip" del gradiente durante la backpropagation in modo che non superi mai i valori di soglia, è molto usato nelle RNN.
In TensorFlow la funzione *minimize()* si occupa di eseguire il clipping al posto nostro, dall'immagine è possibile vedere cosa accade durante il calcolo del gradiente:

![alt text](https://i.gyazo.com/a15e2c281064f74a3ea44c48a0f0b96c.png)